In [2]:
import pandas as pd
import os
from glob import glob
import numpy as np
from pandas.api.types import CategoricalDtype

In [4]:
df_1 = pd.read_parquet('회원_전처리_Segment.parquet')  
df_2 = pd.read_parquet('신용_전처리_Segment.parquet') 
df_3 = pd.read_parquet('승인_전처리_Segment.parquet')
df_4 = pd.read_parquet('청구_전처리_Segment.parquet')
df_5 = pd.read_parquet('잔액_전처리_Segment.parquet')
df_6 = pd.read_parquet('채널_전처리_Segment.parquet')
df_7 = pd.read_parquet('마케팅_전처리_Segment.parquet')
df_8 = pd.read_parquet('성과_전처리_Segment.parquet')

In [5]:
df_merged = (
    df_1
    .merge(df_2,  on=['기준년월','ID','Segment'], how='left')
    .merge(df_3,  on=['기준년월','ID','Segment'], how='left')
    .merge(df_4,  on=['기준년월','ID','Segment'], how='left')
    .merge(df_5,  on=['기준년월','ID','Segment'], how='left')
    .merge(df_6,  on=['기준년월','ID','Segment'], how='left')
    .merge(df_7,  on=['기준년월','ID','Segment'], how='left')
    .merge(df_8, on=['기준년월','ID','Segment'], how='left'))
print(df_merged.shape)
df_merged

(2400000, 52)


,기준년월,ID,Segment,입회일자_신용,수신거부여부_TM,이용카드수_신용체크,이용금액_R3M_신용체크,최종유효년월_신용_이용가능,이용여부_3M_해외겸용_본인,카드이용한도금액,...,잔액_할부_B0M,월중평잔,평잔_일시불_6M,인입일수_ARS_R6M,방문후경과월_앱_R6M,불만제기후경과월_R12M,컨택건수_이용유도_TM_R6M,컨택건수_이용유도_EM_R6M,잔액_신판ca최대한도소진율_r6m,변동률_RV일시불평잔
0,201807,TRAIN_000000,D,20130101,0,1,196,202110.0,0,19354,...,962,17237,2440,8,6,12,3,57,0.849842,0.999998
1,201807,TRAIN_000001,E,20170801,0,1,13475,202112.0,0,9996,...,2390,7967,2677,0,6,12,2,2,0.851009,1.092698
2,201807,TRAIN_000002,C,20080401,0,1,23988,202111.0,0,88193,...,5113,59917,9118,1,0,12,2,12,0.938161,1.006124
3,201807,TRAIN_000003,D,20160501,0,1,3904,202201.0,1,19062,...,5025,27854,884,10,6,12,2,35,1.135424,0.999998
4,201807,TRAIN_000004,E,20180601,0,1,1190,202201.0,1,177222,...,0,0,21,0,6,0,7,0,0.000000,0.999998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,E,20010701,0,1,10755,202110.0,1,20070,...,0,0,0,0,6,0,0,0,0.032439,0.999998
2399996,201812,TRAIN_399996,D,20170701,0,1,27636,202110.0,1,84217,...,0,29429,12524,0,6,12,0,58,0.168081,0.999998
2399997,201812,TRAIN_399997,C,20090501,1,1,23187,202110.0,1,52612,...,2960,7383,3241,0,6,12,0,0,0.190393,0.999998
2399998,201812,TRAIN_399998,E,20130101,1,0,0,202202.0,0,10002,...,0,0,0,0,6,0,0,0,0.012677,0.999998


In [7]:
ex1 = df_merged
ex1 = ex1.fillna(-1)

In [8]:
num_df = ex1.select_dtypes(include=[np.number]).dropna()
low_var = num_df.var()[ num_df.var() < 0.001 ].index.tolist()
low_var

[]

In [9]:
missing_mask = ex1.isna() | (ex1 == -1)
missing_ratio = missing_mask.mean()

high_na = missing_ratio[missing_ratio > 0.2].index.tolist()

high_const_cols = []
threshold_const = 0.8

for col in ex1.columns:
    top_ratio = ex1[col].value_counts(normalize=True, dropna=False).values[0]
    if top_ratio > threshold_const:
        high_const_cols.append(col)

to_drop = list(set(high_na + high_const_cols))

if 'Segment' in to_drop:
    to_drop.remove('Segment')

print("삭제 대상 컬럼 (결측>20% 또는 동일값>80%):", to_drop)

ex1.drop(columns=to_drop, inplace=True)


삭제 대상 컬럼 (결측>20% 또는 동일값>80%): []


In [10]:
import pandas as pd
import numpy as np

num_df = ex1.select_dtypes(include=[np.number]).dropna()

corr = num_df.corr().abs()

high_corr_pairs = (
    corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
        .stack()
        .reset_index()
)
high_corr_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']
high_corr_pairs = high_corr_pairs[high_corr_pairs['Correlation'] > 0.7]

if not np.issubdtype(ex1['Segment'].dtype, np.number):
    segment_map = {label: idx for idx, label in enumerate(sorted(ex1['Segment'].unique()))}
    ex1['Segment_encoded'] = ex1['Segment'].map(segment_map)
else:
    ex1['Segment_encoded'] = ex1['Segment']

segment_corr = ex1[num_df.columns].corrwith(ex1['Segment_encoded']).abs()

high_corr_pairs['Corr_with_Segment_1'] = high_corr_pairs['Feature_1'].map(segment_corr)
high_corr_pairs['Corr_with_Segment_2'] = high_corr_pairs['Feature_2'].map(segment_corr)


high_corr_pairs = high_corr_pairs.sort_values(by='Correlation', ascending=False).reset_index(drop=True)

print(f"▶ 상관계수 0.7 초과 변수쌍 수: {len(high_corr_pairs)}")
display(high_corr_pairs)


▶ 상관계수 0.7 초과 변수쌍 수: 3


,Feature_1,Feature_2,Correlation,Corr_with_Segment_1,Corr_with_Segment_2
0,정상청구원금_B5M,청구금액_R6M,0.876001,0.660738,0.597904
1,이용금액_할부_R12M,잔액_할부_B0M,0.851244,0.353568,0.264292
2,이용금액_R3M_신용체크,정상청구원금_B5M,0.743199,0.622827,0.660738


In [11]:
to_drop = []

for _, row in high_corr_pairs.iterrows():
    f1, f2 = row['Feature_1'], row['Feature_2']
    c1, c2 = row['Corr_with_Segment_1'], row['Corr_with_Segment_2']
    
    if pd.isna(c1) or pd.isna(c2):
        continue
    
    if c1 < c2:
        to_drop.append(f1)
    else:
        to_drop.append(f2)

to_drop = list(set(to_drop))


print(f"▶ 제거 대상 피처 수: {len(to_drop)}")
print("제거할 피처 목록:")
print(to_drop)

▶ 제거 대상 피처 수: 3
제거할 피처 목록:
['청구금액_R6M', '이용금액_R3M_신용체크', '잔액_할부_B0M']


In [12]:
cols_to_drop = ['청구금액_R6M', '이용금액_R3M_신용체크', '잔액_할부_B0M']
ex1.drop(columns=cols_to_drop, inplace=True)

In [13]:
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)


X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)


C:\Users\AHN\AppData\Local\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


VIF > 10인 컬럼들: []
Empty DataFrame
Columns: [feature, VIF]
Index: []


In [17]:
cols_to_drop = ['Segment_encoded']
ex1.drop(columns=cols_to_drop, inplace=True)

In [18]:
ex1.to_parquet('Segment_merge_ver_02.parquet', index=False)

In [19]:
ex1.columns.tolist()

['기준년월',
 'ID',
 'Segment',
 '입회일자_신용',
 '수신거부여부_TM',
 '이용카드수_신용체크',
 '최종유효년월_신용_이용가능',
 '이용여부_3M_해외겸용_본인',
 '카드이용한도금액',
 'CA이자율_할인전',
 '최종이용일자_CA',
 '최종이용일자_체크',
 '최종이용일자_할부',
 '이용후경과월_일시불',
 '이용금액_할부_R12M',
 '이용금액_체크_R12M',
 '최대이용금액_할부_유이자_R12M',
 '쇼핑_기타_이용금액',
 '교통_정비이용금액',
 '교통_택시이용금액',
 '납부_기타이용금액',
 '_1순위업종',
 '_2순위업종',
 '_3순위업종',
 '_2순위쇼핑업종',
 '_3순위쇼핑업종',
 '_1순위교통업종',
 '_2순위교통업종',
 '_1순위납부업종_이용금액',
 'RP건수_B0M',
 'RP금액_B0M',
 'RP후경과월_교통',
 '연체입금원금_B0M',
 '정상청구원금_B5M',
 '연속유실적개월수_기본_24M_카드',
 '청구서발송여부_B0',
 '포인트_적립포인트_R12M',
 '포인트_이용포인트_R3M',
 '할인금액_청구서_R3M',
 '혜택수혜금액_R3M',
 '월중평잔',
 '평잔_일시불_6M',
 '인입일수_ARS_R6M',
 '방문후경과월_앱_R6M',
 '불만제기후경과월_R12M',
 '컨택건수_이용유도_TM_R6M',
 '컨택건수_이용유도_EM_R6M',
 '잔액_신판ca최대한도소진율_r6m',
 '변동률_RV일시불평잔']

In [16]:
ex1

,기준년월,ID,Segment,입회일자_신용,수신거부여부_TM,이용카드수_신용체크,최종유효년월_신용_이용가능,이용여부_3M_해외겸용_본인,카드이용한도금액,CA이자율_할인전,...,월중평잔,평잔_일시불_6M,인입일수_ARS_R6M,방문후경과월_앱_R6M,불만제기후경과월_R12M,컨택건수_이용유도_TM_R6M,컨택건수_이용유도_EM_R6M,잔액_신판ca최대한도소진율_r6m,변동률_RV일시불평잔,Segment_encoded
0,201807,TRAIN_000000,D,20130101,0,1,202110.0,0,19354,22.995207,...,17237,2440,8,6,12,3,57,0.849842,0.999998,3
1,201807,TRAIN_000001,E,20170801,0,1,202112.0,0,9996,14.793821,...,7967,2677,0,6,12,2,2,0.851009,1.092698,4
2,201807,TRAIN_000002,C,20080401,0,1,202111.0,0,88193,22.014276,...,59917,9118,1,0,12,2,12,0.938161,1.006124,2
3,201807,TRAIN_000003,D,20160501,0,1,202201.0,1,19062,22.998014,...,27854,884,10,6,12,2,35,1.135424,0.999998,3
4,201807,TRAIN_000004,E,20180601,0,1,202201.0,1,177222,14.661948,...,0,21,0,6,0,7,0,0.000000,0.999998,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,E,20010701,0,1,202110.0,1,20070,15.243670,...,0,0,0,6,0,0,0,0.032439,0.999998,4
2399996,201812,TRAIN_399996,D,20170701,0,1,202110.0,1,84217,14.843464,...,29429,12524,0,6,12,0,58,0.168081,0.999998,3
2399997,201812,TRAIN_399997,C,20090501,1,1,202110.0,1,52612,17.038599,...,7383,3241,0,6,12,0,0,0.190393,0.999998,2
2399998,201812,TRAIN_399998,E,20130101,1,0,202202.0,0,10002,15.182880,...,0,0,0,6,0,0,0,0.012677,0.999998,4
